In [1]:
import keras_models as kms
import keras_losses as kl
import numpy as np
import metrics as mets

In [2]:
#slp with prior (regularisation)

In [3]:
#could load arbitrary version of x here to get some shape parameters, but cba
#architecture parameters
num_inputs = 13 #aka n in ml terminology
num_outputs = 1 #dimensionality of output y (for single record)
layer_sizes = []
reg_coeffs = [.5, .5]
#propagation parameters
epochs = 1000
m = 506 #total number of records
training_size = 253. / 506.
train_num = int(m * training_size)
batch_num = 1 * train_num
run_ns = range(1,11)

In [4]:
#compile model
model = kms.slp_r(num_inputs, num_outputs, layer_sizes, reg_coeffs)
model.compile(optimizer='adam',
              loss=kl.sum_of_squared_errors,
              metrics=['mean_squared_error'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 14        
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [19]:
for run_n in run_ns:
    #load data
    x_train = np.genfromtxt("../../data/uci/bh_50_x_tr_" + str(run_n) + ".csv", delimiter = ',')
    y_train = np.genfromtxt("../../data/uci/bh_50_y_tr_" + str(run_n) + ".csv", delimiter = ',')

    #train model
    history = model.fit(x_train, y_train,
                        batch_size=batch_num,
                        epochs=epochs,
                        verbose=0)
                            
    #save model
    model.save("../../MPE_examples/saved_keras_models/uci/bh_50_slp_" + str(run_n) + ".h5")

In [ ]:
#load model

In [10]:
MPE_test_losses = []
for run_n in run_ns:
    #load data
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter = ',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter = ',')
    #load keras MPE model
    model = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/uci/bh_50_slp_" + str(run_n) + ".h5", custom_objects={'sum_of_squared_errors': kl.sum_of_squared_errors})
    #calculate evaluation metrics for keras MPE
    y_pred = model.predict(x_test)
    y_test, y_pred = tools.check_dtypes(y_test, y_pred)
    y_test = y_test.reshape(-1,1)
    test_loss = metrics.mean_squared_error(y_test, y_pred) #averaged over test set size
    MPE_test_losses.append(test_loss)

TypeError: sum_of_squared_errors() takes exactly 2 arguments (0 given)

In [ ]:
print MPE_test_losses
print np.mean(MPE_test_losses)
print np.std(MPE_test_losses)